In [ ]:
import pandas as pd
import re
from openpyxl import Workbook

def month_number_to_name(month):
    """Convert month number to abbreviated month name."""
    return ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"][month - 1]

def process_paste_sheet(processed_file_path, current_month, current_year):
    # Load the processed file
    xls = pd.ExcelFile(processed_file_path)
    
    # Create an empty DataFrame to collect all data for the "Paste" sheet
    paste_df = pd.DataFrame(columns=["File Name", "Level 3", "Level 4", "Cost Grouping", "Cost Type", 
                                     "Finance Region", "Attribute Type", "Period", "Cost", 
                                     "FTE/Contractor", "Country", "Level 3.5", "Level 4.5", "Entity", 
                                     "Function", "Mapped Country", "MICA"])

    # Helper function to add a row of data to the paste_df
    def append_data(sheet_name, entity, function, account, attribute_type, period_value, cost_value, fte_value):
        row_data = [sheet_name, '', '', '', '', '', attribute_type, period_value, 
                    cost_value if cost_value != '' else None, 
                    fte_value if fte_value != '' else None, '', '', '', entity, function, '', account]
        paste_df.loc[len(paste_df)] = row_data

    # Process each sheet
    for sheet_name in xls.sheet_names:
        # Only process sheets containing 'Actual', 'Target', or 'Fcst'
        if re.search(r'Actual|Target|Fcst', sheet_name, re.IGNORECASE):
            sheet = pd.read_excel(xls, sheet_name=sheet_name)

            # Extract relevant columns
            entity_col = sheet['Entity'] if 'Entity' in sheet.columns else None
            function_col = sheet['RTN Level 4'] if 'RTN Level 4' in sheet.columns else None
            account_col = sheet['Account'] if 'Account' in sheet.columns else None

            # Identify cost type (DC for Cost, HC for FTE/Contractor)
            cost_type = 'Cost' if 'DC' in sheet_name else 'FTE/Contractor'

            # 1. MoM (Last 3 Months Data)
            for month_col in sheet.columns:
                if re.match(r'M\d{2}', month_col):
                    month_num = int(month_col[1:3])
                    year = int(month_col[3:]) if len(month_col) > 3 else current_year

                    # Check if the month falls within the last 3 months
                    if ((current_month - month_num) % 12 in [0, 1, 2]) and year == current_year:
                        period_value = f"{month_number_to_name(month_num)}-{str(year)[-2:]}"

                        for idx, row in sheet.iterrows():
                            append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                        'MoM', period_value, 
                                        row[month_col] if cost_type == 'Cost' else None, 
                                        row[month_col] if cost_type == 'FTE/Contractor' else None)

            # 2. FY Target
            if 'Target' in sheet_name:
                fy_col = [col for col in sheet.columns if f'FY {current_year}' in col]
                for col in fy_col:
                    period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"

                    for idx, row in sheet.iterrows():
                        append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                    'FY Target', period_value, 
                                    row[col] if cost_type == 'Cost' else None, 
                                    row[col] if cost_type == 'FTE/Contractor' else None)

            # 3. FY Forecast
            if 'Fcst' in sheet_name:
                fy_col = [col for col in sheet.columns if f'FY {current_year}' in col]
                for col in fy_col:
                    period_value = f"Dec-{str(current_year)[-2:]}"

                    for idx, row in sheet.iterrows():
                        append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                    'FY Forecast', period_value, 
                                    row[col] if cost_type == 'Cost' else None, 
                                    row[col] if cost_type == 'FTE/Contractor' else None)

            # 4. YTD (YTD Target, YTD Forecast, YTD Actual)
            ytd_columns = [col for col in sheet.columns if re.match(r'M\d{2}', col) and int(col[1:3]) <= current_month]

            for idx, row in sheet.iterrows():
                ytd_total = row[ytd_columns].sum(skipna=True)

                period_value = f"{month_number_to_name(current_month)}-{str(current_year)[-2:]}"
                if 'Target' in sheet_name:
                    append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                'YTD Target', period_value, 
                                ytd_total if cost_type == 'Cost' else None, 
                                ytd_total if cost_type == 'FTE/Contractor' else None)
                elif 'Fcst' in sheet_name:
                    append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                'YTD Forecast', period_value, 
                                ytd_total if cost_type == 'Cost' else None, 
                                ytd_total if cost_type == 'FTE/Contractor' else None)
                elif 'Actual' in sheet_name:
                    append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                'YTD Actual', period_value, 
                                ytd_total if cost_type == 'Cost' else None, 
                                ytd_total if cost_type == 'FTE/Contractor' else None)

            # 5. Prior Year FY
            prior_year_fy_col = [col for col in sheet.columns if f'FY {current_year - 1}' in col]
            for col in prior_year_fy_col:
                period_value = f"Dec-{str(current_year - 1)[-2:]}"
                for idx, row in sheet.iterrows():
                    append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                'Prior Year FY', period_value, 
                                row[col] if cost_type == 'Cost' else None, 
                                row[col] if cost_type == 'FTE/Contractor' else None)

            # 6. Prior Year YTD
            prior_year_ytd_col = [col for col in sheet.columns if f'YTD {current_year - 1}' in col]
            for col in prior_year_ytd_col:
                period_value = f"{month_number_to_name(current_month)}-{str(current_year - 1)[-2:]}"
                for idx, row in sheet.iterrows():
                    append_data(sheet_name, entity_col[idx], function_col[idx], account_col[idx], 
                                'Prior Year YTD', period_value, 
                                row[col] if cost_type == 'Cost' else None, 
                                row[col] if cost_type == 'FTE/Contractor' else None)

    # Once all data is processed, write the DataFrame to an Excel file
    paste_df.to_excel('final_output_file.xlsx', sheet_name='Paste', index=False)

# Example usage
current_month = 6  # Example: June
current_year = 2024  # Example: Current year
processed_file_path = 'Processed_file.xlsx'  # Update with your file path
process_paste_sheet(processed_file_path, current_month, current_year)
